# HuggingFace Pipeline

In [25]:
%pip install -q transformers datasets       # Huggingface 모델/파이프라인 + 데이터셋 유틸
%pip install -q sentencepiece               # 서브워드 토크나이저(SentencePiece)
%pip install -q kobert-transformers         # koBERT 전용 transformer 래퍼
%pip install -q python-dotenv               # .env (환경변수) 파일

Note: you may need to restart the kernel to use updated packages.


ERROR: Invalid requirement: '#': Expected package name at the start of dependency specifier
    #
    ^


Note: you may need to restart the kernel to use updated packages.


ERROR: Invalid requirement: '#': Expected package name at the start of dependency specifier
    #
    ^


Note: you may need to restart the kernel to use updated packages.


ERROR: Invalid requirement: '#': Expected package name at the start of dependency specifier
    #
    ^


Note: you may need to restart the kernel to use updated packages.


ERROR: Invalid requirement: '#': Expected package name at the start of dependency specifier
    #
    ^


In [26]:
from dotenv import load_dotenv
import os

load_dotenv()
HF_TOKEN = os.getenv('HF_TOKEN')

### NLP Tasks

주어진 task에서 사전 훈련된 모델을 사용하는 가장 간단한 방법은 `pipeline`을 사용하는 것이다.

In [27]:
from transformers import pipeline   # HuggingFace 파이프라인 함수

pipeline() : 항상 Hugging Face Hub에 있는 모델을 기본으로 사용
- 모델을 명시하지 않는 경우
    - Huggingface가 task별 기본(default) 모델을 자동 선택
    - 이 기본모델은 HuggingFace Hub에 공개된 모델
- 모델을 명시한 경우
    - Huggingface에 올라온 특정 repo에서 다운로드
- 모델 객체를 직접 넘길 경우
    - 로컬에서 로드
    - 직접 학습/파인튜닝한 모델
    - 이런 경우에는 pipeline이 "추론 레퍼"의 역할만 한다

##### 기계번역

https://huggingface.co/Helsinki-NLP/opus-mt-ko-en

In [28]:
%pip install tf-keras

Note: you may need to restart the kernel to use updated packages.


In [29]:
# 한국어 -> 영어
translator = pipeline('translation', model='Helsinki-NLP/opus-mt-ko-en')
translator

c:\Users\Playdata\nlp\nlp_venv\Lib\site-packages\transformers\models\marian\tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
Device set to use cpu


In [30]:
translator('핫도그를 향하여 돌격!!!')

[{'translation_text': 'Charge to the hot dog!'}]

In [31]:
translator([
    '공부를 열심히 하시면 좋겠어요.',
    '수업을 열심히 들으시면 좋겠고요.',
    '실습도 열심히 해주시면 좋겠어요.',
    '코딩테스트도 까먹지말고 열심히 합시다.',
    '모두 성공합시다!'
])

[{'translation_text': 'I hope you study hard.'},
 {'translation_text': "I'd like you to take the class really hard."},
 {'translation_text': "I'd like you to work hard on your practice."},
 {'translation_text': "Let's work hard not to forget the coding test."},
 {'translation_text': "Let's all succeed!"}]

##### 감성 분석

In [32]:
sentiment_clf = pipeline('sentiment-analysis')

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use cpu


In [33]:
sentiment_clf("I'm the worst and I like it")

[{'label': 'NEGATIVE', 'score': 0.8970896601676941}]

- 한국어

https://huggingface.co/sangrimlee/bert-base-multilingual-cased-nsmc

In [34]:
ko_sentiment_clf = pipeline('sentiment-analysis', model='sangrimlee/bert-base-multilingual-cased-nsmc')

Device set to use cpu


In [35]:
ko_sentiment_clf([
    '어제 배운 Attention이 어려워서 맥주를 한 잔 했어.',
    '공부는 어렵지만 맥주는 시원하더라',
    '하지만 나는 굴하지 않고 열심히 공부할거야!!!'
])

[{'label': 'negative', 'score': 0.9047993421554565},
 {'label': 'negative', 'score': 0.7013086676597595},
 {'label': 'positive', 'score': 0.7580994367599487}]

##### Zero Shot Classification

- shot == 예시
- 예시 없이 (학습 없이) 추론(분류)

In [ ]:
# 제로샷 분류 파이프라인 (기본 NLI 모델 자동 로드)
zero_shot_clf = pipeline('zero-shot-classification')

No model was supplied, defaulted to facebook/bart-large-mnli and revision d7645e1 (https://huggingface.co/facebook/bart-large-mnli).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use cpu


In [37]:
zero_shot_clf(
    "This is a course about the transformers library.",
    candidate_labels=["education", "politics", "business"]
)

{'sequence': 'This is a course about the transformers library.',
 'labels': ['education', 'business', 'politics'],
 'scores': [0.9053581953048706, 0.07259626686573029, 0.022045595571398735]}

In [38]:
zero_shot_clf(
    "It has a poison and it's very dangerous",
    candidate_labels=["tiger", "squirrel", "snake"]
)

{'sequence': "It has a poison and it's very dangerous",
 'labels': ['snake', 'tiger', 'squirrel'],
 'scores': [0.6373922228813171, 0.19870398938655853, 0.16390381753444672]}

- 한국어

https://huggingface.co/joeddav/xlm-roberta-large-xnli

In [ ]:
# 다국어(XNLI) 기반 제로샷 분류모델
ko_zero_shot_clf = pipeline('zero-shot-classification', model='joeddav/xlm-roberta-large-xnli')

c:\Users\Playdata\nlp\nlp_venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Playdata\.cache\huggingface\hub\models--joeddav--xlm-roberta-large-xnli. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular

In [ ]:
sequence_to_classify = "2025년에는 어떤 운동을 하시겠습니까?!"  # 분류할 입력 문장
candidate_labels = ["정치", "경제", "건강"]                     # 비교할 후보 라벨

ko_zero_shot_clf(sequence_to_classify, candidate_labels)

{'sequence': '2025년에는 어떤 운동을 하시겠습니까?!',
 'labels': ['정치', '건강', '경제'],
 'scores': [0.40395545959472656, 0.3669034242630005, 0.22914111614227295]}

In [ ]:
sequence_to_classify = "2025년에는 어떤 운동을 하시겠습니까?!"
candidate_labels = ["정치", "경제", "건강"]
hypothesis_template = "이 텍스트는 {}에 관한 내용입니다."           # NLI 가설 문장 템플릿

ko_zero_shot_clf(sequence_to_classify, candidate_labels, hypothesis_template=hypothesis_template)

{'sequence': '2025년에는 어떤 운동을 하시겠습니까?!',
 'labels': ['건강', '정치', '경제'],
 'scores': [0.9117788672447205, 0.05924064293503761, 0.02898053266108036]}

##### 텍스트 생성

In [42]:
text_generator = pipeline('text-generation', model='distilgpt2')

c:\Users\Playdata\nlp\nlp_venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Playdata\.cache\huggingface\hub\models--distilgpt2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For b

In [ ]:
text_generator(
    "In this course, we will teach you how to", # 텍스트 생성 시작 프롬프트
    max_length=30,                              # 생성 결과 최대 토큰 길이
    num_return_sequences=2                      # 서로 다른 생성 결과 2개 반환
)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=256) and `max_length`(=30) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


[{'generated_text': 'In this course, we will teach you how to use the power of the word “” to describe the world in a way that only the world can understand.\n\n\n\nThe purpose of this course is to demonstrate the importance of using the word “” to describe the world in a way that only the world can understand.\nIn this course we will cover how to use the word “” to describe the world in a way that only the world can understand.\nIn this course we will show you how to use the word “” to describe the world in a way that only the world can understand.\nIn this course we will show you how to use the word “” to describe the world in a way that only the world can understand.\nIn this course we will show you how to use the word “” to describe the world in a way that only the world can understand.\nIn this course we will show you how to use the word “” to describe the world in a way that only the world can understand.\nIn this course we will show you how to use the word “” to describe the wor

- 한국어

https://huggingface.co/skt/ko-gpt-trinity-1.2B-v0.5

In [44]:
ko_text_generator = pipeline('text-generation', model='skt/ko-gpt-trinity-1.2B-v0.5')

c:\Users\Playdata\nlp\nlp_venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Playdata\.cache\huggingface\hub\models--skt--ko-gpt-trinity-1.2B-v0.5. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular H

In [56]:
ko_text_generator('AI란 말입니다')
# ko_text_generator('오늘 점심에는')

[{'generated_text': "AI란 말입니다' 같은 문구는 '개인정보'라는 말에 힘을 싣고 있습니다. 개인정보란 말 그대로 개인의 모든 정보를 말합니다. 따라서 개인정보를 관리하고 제공하는 사람은 당연히 개인 정보를 보호해야 하고 그 정보를 제공받을 권리도 당연히 따라야 한다는 것이지요. 이런 맥락에서 '정보통신망 이용촉진 및 정보보호 등에 관한 법률'은 정보통신서비스 제공자에게 수집된 개인정보의 보호조치를 취하도록 하고 있습니다. \n 그렇다면 '개인정보'라는 말은 언제부터 사용하게 됐을까요. 우리말로 하자면 '신용카드'와 '주민등록번호' 정도가 될 것 같습니다. '정보통신망 이용촉진 및 정보보호 등에 관한 법률'에서는 '개인정보'란 단어를 '개인정보주체에게 제공된 전체 또는 부분'이라고 정의하고 있습니다. 한마디로 '개인정보'란 말은 신용카드를 비롯해 각종 금융서비스, 주민등록번호, 휴대전화번호와 같이 정보통신서비스를 통해 제공된 모든 정보를 말합니다. \n 그런데 여기서 짚고 넘어갈 것이 있습니다. '개인정보'란 단어에는 그 단어가 주는 뉘앙스가 있습니다. '신용카드'나 '주민등록번호'는 신용을 상징하는 단어입니다. 그런데 '개인정보'란 단어는 신용카드를 비롯해 금융서비스 등 각종 금융서비스를 통해 제공되는 모든 정보를 말합니다. 따라서 '개인정보'란 단어는 신용카드를 비롯해 금융서비스 등 각종 금융서비스를 통해 제공된 모든 정보를 말합니다. \n 그러면"}]

##### Fill Mask

In [ ]:
unmasker = pipeline('fill-mask')

No model was supplied, defaulted to distilbert/distilroberta-base and revision fb53ab8 (https://huggingface.co/distilbert/distilroberta-base).
Using a pipeline without specifying a model name and revision in production is not recommended.
c:\Users\Playdata\nlp\nlp_venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Playdata\.cache\huggingface\hub\models--distilbert--distilroberta-base. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, se

In [57]:
unmasker('This course will teach you all about <mask> model.', top_k=3)

[{'score': 0.09073042869567871,
  'token': 265,
  'token_str': ' business',
  'sequence': 'This course will teach you all about business model.'},
 {'score': 0.072625532746315,
  'token': 30412,
  'token_str': ' mathematical',
  'sequence': 'This course will teach you all about mathematical model.'},
 {'score': 0.049304164946079254,
  'token': 5,
  'token_str': ' the',
  'sequence': 'This course will teach you all about the model.'}]

##### NER

In [ ]:
ner = pipeline(
    'ner', 
    model='dslim/bert-base-NER', 
    grouped_entities=True       # 연속된 동일 개체를 하나로 묶어서 반환
)

c:\Users\Playdata\nlp\nlp_venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Playdata\.cache\huggingface\hub\models--dslim--bert-base-NER. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP downl

In [58]:
ner('My name is Squirrel and I work at Playdata in Seoul')

[{'entity_group': 'PER',
  'score': np.float32(0.98995215),
  'word': 'S',
  'start': 11,
  'end': 12},
 {'entity_group': 'PER',
  'score': np.float32(0.46767735),
  'word': '##qui',
  'start': 12,
  'end': 15},
 {'entity_group': 'PER',
  'score': np.float32(0.72662497),
  'word': '##rrel',
  'start': 15,
  'end': 19},
 {'entity_group': 'ORG',
  'score': np.float32(0.986338),
  'word': 'Playdata',
  'start': 34,
  'end': 42},
 {'entity_group': 'LOC',
  'score': np.float32(0.9994649),
  'word': 'Seoul',
  'start': 46,
  'end': 51}]

##### Q&A

In [50]:
qna = pipeline('question-answering')

No model was supplied, defaulted to distilbert/distilbert-base-cased-distilled-squad and revision 564e9b5 (https://huggingface.co/distilbert/distilbert-base-cased-distilled-squad).
Using a pipeline without specifying a model name and revision in production is not recommended.
c:\Users\Playdata\nlp\nlp_venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Playdata\.cache\huggingface\hub\models--distilbert--distilbert-base-cased-distilled-squad. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as 

In [51]:
qna(
    question="Where do I work?",
    context="My name is Squirrel and I work at Playdata in Seoul"
)

{'score': 0.8710189713747241, 'start': 34, 'end': 42, 'answer': 'Playdata'}

- 한국어

In [52]:
ko_qna = pipeline('question-answering', model='klue/roberta-base')

c:\Users\Playdata\nlp\nlp_venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Playdata\.cache\huggingface\hub\models--klue--roberta-base. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP downloa

In [53]:
ko_qna(
    question="1931년 어떤 진단을 받았나요?",
    context="""그러던 1931년, 그는 갑작스럽게 폐결핵을 진단받았다. 이상은 하루에 담배를 50개피 이상 피는 것을 자신의 일과라고 표현했을 정도로 엄청난 골초였는데, 그 때문인지 병세는 날이 갈수록 악화되어 담당의가 그의 폐를 확인하고는 형체도 안 보인다며 혀를 내둘렀을 정도였다. 결국 1933년부터는 각혈까지 시작되었고, 건축기사 일을 지속하기 어렵다고 판단한 이상은 조선총독부에서 퇴사하고 황해도에 있는 배천 온천으로 요양을 간다."""
)

{'score': 0.0002554136735852808,
 'start': 155,
 'end': 168,
 'answer': '1933년부터는 각혈까지'}

In [54]:
help(ko_qna)

Help on QuestionAnsweringPipeline in module transformers.pipelines.question_answering object:

class QuestionAnsweringPipeline(transformers.pipelines.base.ChunkPipeline)
 |  QuestionAnsweringPipeline(model: Union[ForwardRef('PreTrainedModel'), ForwardRef('TFPreTrainedModel')], tokenizer: transformers.tokenization_utils.PreTrainedTokenizer, modelcard: Optional[transformers.modelcard.ModelCard] = None, framework: Optional[str] = None, task: str = '', **kwargs)
 |
 |  Question Answering pipeline using any `ModelForQuestionAnswering`. See the [question answering
 |  examples](../task_summary#question-answering) for more information.
 |
 |  Example:
 |
 |  ```python
 |  >>> from transformers import pipeline
 |
 |  >>> oracle = pipeline(model="deepset/roberta-base-squad2")
 |  >>> oracle(question="Where do I live?", context="My name is Wolfgang and I live in Berlin")
 |  {'score': 0.9191, 'start': 34, 'end': 40, 'answer': 'Berlin'}
 |  ```
 |
 |  Learn more about the basics of using a pipeli